### Packages needed

In [3]:
import numpy as np
import pandas as pd
import rasterio
import fiona
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.plot import show
import rasterio.mask
import datetime 
import gdal

###  Functions

In [4]:
def CreateRaster(lon,lat,data,resx,resy,save_directory):
    from osgeo import gdal
    from osgeo import gdal_array
    from osgeo import osr,ogr

    xmin,ymin,xmax,ymax = [lon.min(),lat.min(),lon.max(),lat.max()]
    nrows,ncols = np.shape(data)
    geotransform=(xmin-resy,resy,0,ymax+resx,0, -resx)  

    output_raster = gdal.GetDriverByName('GTiff').Create(save_directory,ncols, nrows, 1 ,gdal.GDT_Float32)  # Open the file

    output_raster.SetGeoTransform(geotransform)  # Specify its coordinates
    srs = osr.SpatialReference()                 # Establish its coordinate encoding
    srs.ImportFromEPSG(4326)                     # This one specifies WGS84 lat long.

                                                 # Anyone know how to specify the 

                                                 # IAU2000:49900 Mars encoding
    output_raster.SetProjection( srs.ExportToWkt() )   # Exports the coordinate system                                         
    output_raster.GetRasterBand(1).WriteArray(data)   # Writes my array to the raster
    output_raster.FlushCache()
    return

### Creation of  Aridity rasters

In [5]:
path='D:/UNBC/HydroMet_Project/Data/ERA5_land/'

for year in np.arange(1981,1982):
    A=[];PET=[];Pre=[]
    for month in np.arange(1,13):
        #resampled  precipitation raster
        with rasterio.open(path+'Tot_Precip/Monthly_Rasters/%s_%s_RE.tif'%(year,month) ) as src:
            Precip= src.read(1)
            out_meta = src.meta.copy()
            
        #resampled PET raster
        with rasterio.open(path+'Potential_Evap/Raster_Calc_PET/%s_%s_Min_Ham.tif'%(year,month) ) as src:
            PE = src.read(1)
            out_meta = src.meta.copy()
            
        # difference between precip  and PET
        A.append(PE-Precip)
      
    #Create Aridity Array
    A=np.array(A)
    Shape=np.shape(A)
    Aridity=np.empty([Shape[1],Shape[2]])
    for i in range(Shape[1]):
        for j in range(Shape[2]):
            W=A[:,i,j]
            
            # calculate annual aridity
            Aridity[i,j]=np.sum(W[W>0])
            
            
    #open lat and long coordinates
    lat=np.load(path+'Potential_Evap/Lat_NEW.npy')
    long=np.load(path+'Potential_Evap/Long_NEW.npy')
    
    #get x and y resolution
    Resx=lat[0][0]-lat[1][0]
    Resy=lat[0][0]-lat[1][0]
    

    x= np.array(Aridity)

    #where to save the raster to
    Save_file=path+'Aridity/%s_Ham_Min.tif'%(year)

#     reshape data dimensions
    data_array_ERA5=x
    lat_array_ERA5=lat
    lon_array_ERA5=long

    # Create the raster
    CreateRaster(lon=lon_array_ERA5,lat=lat_array_ERA5,data=data_array_ERA5, resx=Resx, resy=Resy, save_directory=Save_file)

        